In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [35]:

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

In [36]:
labels = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
nelabels = ['Residential', 'NonResidential']
img_size = 64
#data_dir = '../input/eurosat-dataset/EuroSAT'
def get_data(data_dir, str1):
    data = [] 
    if str1 == 'train':
        thresh = 300
    else:
        thresh = 150
    for label in labels: 
        path = os.path.join(data_dir, label)
        if label == 'Industrial' or label == 'Residential':
            class_num = 0
            #thresh = 1000
        else:
            class_num = 1
        #class_num = labels.index(label)
        counter = 0
        for img in os.listdir(path):
            counter = counter + 1
            if counter > thresh:
                break
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                if resized_arr.shape[0] != 64:
                    continue
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [103]:
import os

In [59]:
#Now we can easily fetch our train and validation data.
train = get_data('../input/eurosat-dataset/EuroSAT', 'train')
val = get_data('../input/eurosat-dataset/EuroSAT', 'val')

In [60]:
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in val:
  x_val.append(feature)
  y_val.append(label)

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

print(len(x_train))
print(len(y_train))

In [61]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [98]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(64,64,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

In [43]:
opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [112]:
#testImage = cv2.imread("../input/bangalore/bangalore00.jpg")
files = os.listdir("../input/bangalore-segment/")
testArr = []
for file in files:
    print("IN")
    if not os.path.exists(os.path.join("../input/bangalore-segment/",file)):
        continue
    
    testImage = cv2.imread(file)
    testImage = cv2.resize(testImage, (64, 64))
    testArr.append(testImage)
#testImage = cv2.resize(testImage, (64, 64))
#val_acc = history.history['val_accuracy']
#print(val_acc)
#acc = history.history['accuracy']
#print(acc)
model.save('../working/base_classification.h5')
x_val = testArr
x_val = np.array(x_val) / 255
x_val.reshape(-1, img_size, img_size, 1)
predictions = model.predict(x_val)
predictions = predictions.reshape(1,-1)[0]
print(len(x_val), len(y_val))
#print(classification_report(y_val, predictions, target_names = ['Res (Class 0)','NonRes (Class 1)']))
print(predictions)
if predictions[0] > predictions[1] :
    print('Residential')
else:
    print('Non - Residential')

In [67]:
history = model.fit(x_train,y_train,epochs = 50, validation_data = (x_val, y_val))